In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import timedelta

In [2]:
lookback_period = 50
start_date = pd.to_datetime('2019-11-01')-timedelta(days=lookback_period) # start from 2019-11-01, but previous 50 tracking data 
end_date = '2024-10-31'

btc_data = pd.read_csv('Gemini_BTCUSD_d.csv', skiprows=1)
btc_data = btc_data.iloc[:, 1:]
btc_data['date'] = pd.to_datetime(btc_data['date']).dt.date
btc_data = btc_data.sort_values('date', ascending=True)

rf_data = pd.read_csv('Risk_Free.csv')
rf_data['date'] = pd.to_datetime(rf_data['date']).dt.date

spy_data = yf.download('SPY', start=start_date, end=end_date)
spy_data = spy_data.reset_index()
spy_data.columns = spy_data.columns.get_level_values(0) 
spy_data['date'] = spy_data['Date'].dt.date
spy_data = spy_data[['date', 'Adj Close']] 
spy_data.rename(columns={'Adj Close': 'SPY'}, inplace=True)

data = pd.merge(btc_data, rf_data, on='date', how='inner')
data = data.merge(spy_data, on='date', how='inner')
data = data.sort_values(by='date').reset_index(drop=True)
data


[*********************100%***********************]  1 of 1 completed


,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf,SPY
0,2019-09-12,BTC/USD,10110.81,10465.63,10033.92,10393.16,780.113328,8.107843e+06,0.000062,277.620209
1,2019-09-13,BTC/USD,10393.16,10430.08,10156.59,10313.00,841.568950,8.679101e+06,0.000062,277.435913
2,2019-09-16,BTC/USD,10316.55,10351.86,10087.50,10250.00,1181.005973,1.210531e+07,0.000062,276.578979
3,2019-09-17,BTC/USD,10250.00,10282.91,10139.73,10213.55,603.691178,6.165830e+06,0.000062,277.279205
4,2019-09-18,BTC/USD,10213.55,10231.90,9592.50,9814.55,2706.740726,2.656544e+07,0.000055,277.445099
...,...,...,...,...,...,...,...,...,...,...
1531,2024-10-24,BTC/USD,66614.36,68788.12,66446.18,68144.29,754.270437,5.139922e+07,0.000137,579.239990
1532,2024-10-25,BTC/USD,68144.29,68721.97,65525.72,66593.95,1174.413666,7.820884e+07,0.000137,579.039978
1533,2024-10-28,BTC/USD,67919.20,70200.00,67529.77,69924.01,1174.466157,8.212338e+07,0.000137,580.830017
1534,2024-10-29,BTC/USD,69924.01,73646.23,69716.05,72700.31,1533.733035,1.115029e+08,0.000137,581.770020


In [3]:
# create the signal for the trading range breakout

data['SUP'] = data['close'].shift(1).rolling(window=lookback_period).min()
data['RES'] = data['close'].shift(1).rolling(window=lookback_period).max()
data

,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf,SPY,SUP,RES
0,2019-09-12,BTC/USD,10110.81,10465.63,10033.92,10393.16,780.113328,8.107843e+06,0.000062,277.620209,NaN,NaN
1,2019-09-13,BTC/USD,10393.16,10430.08,10156.59,10313.00,841.568950,8.679101e+06,0.000062,277.435913,NaN,NaN
2,2019-09-16,BTC/USD,10316.55,10351.86,10087.50,10250.00,1181.005973,1.210531e+07,0.000062,276.578979,NaN,NaN
3,2019-09-17,BTC/USD,10250.00,10282.91,10139.73,10213.55,603.691178,6.165830e+06,0.000062,277.279205,NaN,NaN
4,2019-09-18,BTC/USD,10213.55,10231.90,9592.50,9814.55,2706.740726,2.656544e+07,0.000055,277.445099,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1531,2024-10-24,BTC/USD,66614.36,68788.12,66446.18,68144.29,754.270437,5.139922e+07,0.000137,579.239990,53951.3,68416.32
1532,2024-10-25,BTC/USD,68144.29,68721.97,65525.72,66593.95,1174.413666,7.820884e+07,0.000137,579.039978,53951.3,68416.32
1533,2024-10-28,BTC/USD,67919.20,70200.00,67529.77,69924.01,1174.466157,8.212338e+07,0.000137,580.830017,53951.3,68416.32
1534,2024-10-29,BTC/USD,69924.01,73646.23,69716.05,72700.31,1533.733035,1.115029e+08,0.000137,581.770020,53951.3,69924.01


In [6]:
data.head(51)

,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf,SPY,SUP,RES
0,2019-09-12,BTC/USD,10110.81,10465.63,10033.92,10393.16,780.113328,8.107843e+06,0.000062,277.620209,NaN,NaN
1,2019-09-13,BTC/USD,10393.16,10430.08,10156.59,10313.00,841.568950,8.679101e+06,0.000062,277.435913,NaN,NaN
2,2019-09-16,BTC/USD,10316.55,10351.86,10087.50,10250.00,1181.005973,1.210531e+07,0.000062,276.578979,NaN,NaN
3,2019-09-17,BTC/USD,10250.00,10282.91,10139.73,10213.55,603.691178,6.165830e+06,0.000062,277.279205,NaN,NaN
4,2019-09-18,BTC/USD,10213.55,10231.90,9592.50,9814.55,2706.740726,2.656544e+07,0.000055,277.445099,NaN,NaN
5,2019-09-19,BTC/USD,9814.55,9814.55,9814.55,9814.55,0.000000,0.000000e+00,0.000055,277.426697,NaN,NaN
6,2019-09-20,BTC/USD,10238.95,10238.95,10057.55,10129.84,895.993225,9.076268e+06,0.000055,276.115906,NaN,NaN
7,2019-09-23,BTC/USD,9974.22,9985.24,9593.00,9705.64,2191.399033,2.126893e+07,0.000055,276.051147,NaN,NaN
8,2019-09-24,BTC/USD,9705.64,9776.27,7991.18,8580.00,9827.467146,8.431967e+07,0.000055,273.884979,NaN,NaN
9,2019-09-25,BTC/USD,8580.00,8652.00,8221.21,8368.01,2601.315039,2.176783e+07,0.000055,275.504974,NaN,NaN


In [4]:
data = data.dropna()
data = data.reset_index(drop=True)

data['signal'] = np.where(data['close'] > data['RES'], 1, 0)  # Buy signal
data['signal'] = np.where(data['close'] < data['SUP'], -1, data['signal'])  # Sell signal

data

,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf,SPY,SUP,RES,signal
0,2019-11-21,BTC/USD,8106.03,8106.03,8106.03,8106.03,0.000000,0.000000e+00,0.000048,287.214935,7420.27,10393.16,0
1,2019-11-22,BTC/USD,7587.78,7718.29,6789.00,7292.60,3685.697145,2.687831e+07,0.000048,287.853729,7420.27,10313.00,-1
2,2019-11-25,BTC/USD,6682.52,7376.90,6523.92,7241.86,2419.475732,1.752150e+07,0.000048,290.084625,7292.60,10250.00,-1
3,2019-11-26,BTC/USD,7241.86,7338.99,7028.27,7073.30,1109.095498,7.844965e+06,0.000048,290.741821,7241.86,10213.55,-1
4,2019-11-27,BTC/USD,7073.30,7669.56,6849.95,7507.18,1975.763120,1.483241e+07,0.000048,292.037842,7073.30,10129.84,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,2024-10-24,BTC/USD,66614.36,68788.12,66446.18,68144.29,754.270437,5.139922e+07,0.000137,579.239990,53951.30,68416.32,0
1482,2024-10-25,BTC/USD,68144.29,68721.97,65525.72,66593.95,1174.413666,7.820884e+07,0.000137,579.039978,53951.30,68416.32,0
1483,2024-10-28,BTC/USD,67919.20,70200.00,67529.77,69924.01,1174.466157,8.212338e+07,0.000137,580.830017,53951.30,68416.32,1
1484,2024-10-29,BTC/USD,69924.01,73646.23,69716.05,72700.31,1533.733035,1.115029e+08,0.000137,581.770020,53951.30,69924.01,1


## Testing

In [5]:
# Initialize simulation variables
cb = 0.03  # Buy commission
cs = 0.03  # Sell commission
T = len(data)  # Total number of time steps
nUSD = 1.0  # Initial USD balance
nBTC = 0.0  # Initial BTC balance
CR = [1.0]  # Cumulative return (start with 1)

# Trading simulation loop
for t in range(T - 1):
    # Get the signal for the current time step
    signal = data.loc[t, 'signal']
    price_t = data.loc[t, 'close']
    price_t1 = data.loc[t + 1, 'close']
    
    if signal == 1 and nBTC == 0:  # Buy condition
        nBTC = nUSD * (1 - cb) / price_t
        nUSD = 0
    elif signal == -1 and nBTC > 0:  # Sell condition
        nUSD = nBTC * (1 - cs) * price_t
        nBTC = 0
    elif signal == -1 and nBTC == 0:  # Short condition
        nBTCb = nUSD / price_t
        nUSD = nUSD + nBTCb * (1 - cs) * price_t - nBTCb * price_t1 / (1 - cb)
    
    # Update cumulative return
    CR.append(nUSD + nBTC * price_t1 * (1 - cs))

# Save cumulative return to the data frame
data['cumulative_return'] = CR

In [6]:
data

,date,symbol,open,high,low,close,Volume BTC,Volume USD,rf,SPY,SUP,RES,signal,cumulative_return
0,2019-11-21,BTC/USD,8106.03,8106.03,8106.03,8106.03,0.000000,0.000000e+00,0.000048,287.214935,7420.27,10393.16,0,1.000000
1,2019-11-22,BTC/USD,7587.78,7718.29,6789.00,7292.60,3685.697145,2.687831e+07,0.000048,287.853729,7420.27,10313.00,-1,1.000000
2,2019-11-25,BTC/USD,6682.52,7376.90,6523.92,7241.86,2419.475732,1.752150e+07,0.000048,290.084625,7292.60,10250.00,-1,0.946245
3,2019-11-26,BTC/USD,7241.86,7338.99,7028.27,7073.30,1109.095498,7.844965e+06,0.000048,290.741821,7241.86,10213.55,-1,0.911298
4,2019-11-27,BTC/USD,7073.30,7669.56,6849.95,7507.18,1975.763120,1.483241e+07,0.000048,292.037842,7073.30,10129.84,0,0.798146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1481,2024-10-24,BTC/USD,66614.36,68788.12,66446.18,68144.29,754.270437,5.139922e+07,0.000137,579.239990,53951.30,68416.32,0,0.040408
1482,2024-10-25,BTC/USD,68144.29,68721.97,65525.72,66593.95,1174.413666,7.820884e+07,0.000137,579.039978,53951.30,68416.32,0,0.039489
1483,2024-10-28,BTC/USD,67919.20,70200.00,67529.77,69924.01,1174.466157,8.212338e+07,0.000137,580.830017,53951.30,68416.32,1,0.041463
1484,2024-10-29,BTC/USD,69924.01,73646.23,69716.05,72700.31,1533.733035,1.115029e+08,0.000137,581.770020,53951.30,69924.01,1,0.043110
